In [2]:
# Install required packages
!pip install transformers torch Pillow pandas numpy sentence-transformers  python-dotenv
!pip install datasets --quiet  # HuggingFace datasets library
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
pip install python-dotenv

In [4]:
# Import libraries
import pandas as pd
import numpy as np
from PIL import Image
import requests
import gradio as gr
from dotenv import load_dotenv
import warnings

In [5]:
# Machine Learning & NLP
from sentence_transformers import SentenceTransformer
from transformers import (
    pipeline,
    Blip2Processor,
    Blip2ForConditionalGeneration,
    AutoTokenizer,
    AutoModelForSeq2SeqLM
)

In [6]:
# Suppress warnings
warnings.filterwarnings('ignore')
load_dotenv()

False

In [7]:
from datasets import load_dataset

# Load USDA FoodData Central dataset
try:
    usda_dataset = load_dataset("goendalf666/usda-food-data-central")
    nutrition_df = pd.DataFrame(usda_dataset['train'])
except:
    # Fallback to sample data if dataset unavailable
    print("Using sample dataset as USDA dataset unavailable")
    nutrition_df = pd.DataFrame({
        'description': ['Apple raw', 'Banana raw', 'Chicken breast roasted', 'Salmon baked', 'Broccoli raw'],
        'foodCategory': ['Fruits', 'Fruits', 'Protein Foods', 'Protein Foods', 'Vegetables'],
        'servingSize': [100, 100, 100, 100, 100],
        'servingSizeUnit': ['g', 'g', 'g', 'g', 'g'],
        'foodNutrients': [
            [{'nutrientName': 'Energy', 'value': 52}, {'nutrientName': 'Protein', 'value': 0.3},
             {'nutrientName': 'Carbohydrate', 'value': 14}, {'nutrientName': 'Total fat', 'value': 0.2}],
            [{'nutrientName': 'Energy', 'value': 89}, {'nutrientName': 'Protein', 'value': 1.1},
             {'nutrientName': 'Carbohydrate', 'value': 23}, {'nutrientName': 'Total fat', 'value': 0.3}],
            [{'nutrientName': 'Energy', 'value': 165}, {'nutrientName': 'Protein', 'value': 31},
             {'nutrientName': 'Carbohydrate', 'value': 0}, {'nutrientName': 'Total fat', 'value': 3.6}],
            [{'nutrientName': 'Energy', 'value': 208}, {'nutrientName': 'Protein', 'value': 20.4},
             {'nutrientName': 'Carbohydrate', 'value': 0}, {'nutrientName': 'Total fat', 'value': 13.4}],
            [{'nutrientName': 'Energy', 'value': 55}, {'nutrientName': 'Protein', 'value': 3.7},
             {'nutrientName': 'Carbohydrate', 'value': 11}, {'nutrientName': 'Total fat', 'value': 0.6}]
        ]
    })

# Process nutrition data into a more usable format
def extract_nutrients(row):
    nutrients = {}
    # Access the 'foodNutrients' column of the row
    for item in row:  # Changed from row['foodNutrients'] to row
        name = item['nutrientName'].lower()
        if 'energy' in name:
            nutrients['calories'] = item['value']
        elif 'protein' in name:
            nutrients['protein'] = item['value']
        elif 'carbohydrate' in name:
            nutrients['carbs'] = item['value']
        elif 'fat' in name:
            nutrients['fats'] = item['value']
    return pd.Series(nutrients)

# Apply the function to the 'foodNutrients' column and concatenate results
nutrition_df = pd.concat([nutrition_df, nutrition_df['foodNutrients'].apply(lambda x: extract_nutrients(x))], axis=1) # use lambda to pass each element to the extraction function
nutrition_df['allergens'] = 'none'  # Simplified for demo

Using sample dataset as USDA dataset unavailable


In [8]:
# Initialize BLIP-2 for food recognition (more accurate than BLIP-1)
blip_processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
blip_model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", device_map="auto")

# Initialize nutrition explanation model (Flan-T5)
nutrition_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
nutrition_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Initialize food similarity model
food_encoder = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize text generation for explanations
explanation_generator = pipeline("text-generation", model="gpt2")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [9]:
def recognize_food_from_image(image):
    """Identify food items from an image using BLIP-2"""
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    inputs = blip_processor(image, return_tensors="pt").to("cuda")
    generated_ids = blip_model.generate(**inputs)
    caption = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    return caption

# %%
def get_nutritional_info(food_name):
    """Retrieve nutritional information for a food item"""
    # Encode food name for similarity search
    food_embedding = food_encoder.encode(food_name.lower())

    # Calculate similarities
    nutrition_df['similarity'] = nutrition_df['description'].apply(
        lambda x: food_encoder.encode(x.lower()) @ food_embedding.T
    )

    best_match = nutrition_df.loc[nutrition_df['similarity'].idxmax()]

    return {
        'food_name': best_match['description'],
        'category': best_match['foodCategory'],
        'calories': best_match.get('calories', 0),
        'protein': best_match.get('protein', 0),
        'carbs': best_match.get('carbs', 0),
        'fats': best_match.get('fats', 0),
        'allergens': best_match['allergens']
    }

# %%
def generate_meal_plan(user_profile):
    """Generate personalized meal plan"""
    # Define dietary guidelines
    guidelines = {
        'weight_loss': {'calories': 1500, 'protein': 75, 'carbs': 150, 'fats': 50},
        'muscle_gain': {'calories': 2500, 'protein': 150, 'carbs': 300, 'fats': 80},
        'maintenance': {'calories': 2000, 'protein': 50, 'carbs': 250, 'fats': 70},
        'diabetic': {'calories': 1800, 'protein': 60, 'carbs': 150, 'fats': 60}
    }

    # Get base targets
    targets = guidelines.get(user_profile['health_goal'], guidelines['maintenance']).copy()

    # Adjust for activity level
    targets = {k: v * user_profile['activity_level'] for k, v in targets.items()}

    # Filter foods based on preferences
    suitable_foods = nutrition_df.copy()
    if user_profile['dietary_preferences'] == 'vegetarian':
        suitable_foods = suitable_foods[~suitable_foods['foodCategory'].str.contains('Protein Foods|Meat')]
    elif user_profile['dietary_preferences'] == 'vegan':
        suitable_foods = suitable_foods[~suitable_foods['foodCategory'].str.contains('Dairy|Protein Foods|Meat')]

    # Filter allergens
    if user_profile['allergies'] != 'none':
        suitable_foods = suitable_foods[~suitable_foods['allergens'].str.contains(user_profile['allergies'])]

    # Generate meal plan (simplified algorithm)
    meal_plan = {}
    meals = ['breakfast', 'lunch', 'dinner']
    proportions = [0.25, 0.35, 0.4]  # Distribution of nutrients

    for meal, proportion in zip(meals, proportions):
        meal_targets = {k: v * proportion for k, v in targets.items()}

        # Select foods that fit the targets
        selected_foods = []
        remaining = meal_targets.copy()

        # Prioritize protein sources first
        protein_foods = suitable_foods.sort_values('protein', ascending=False)
        for _, food in protein_foods.iterrows():
            if remaining['protein'] <= 0:
                break
            if food['protein'] > 0:
                selected_foods.append(food['description'])
                remaining['protein'] -= food['protein']
                remaining['calories'] -= food.get('calories', 0)
                remaining['carbs'] -= food.get('carbs', 0)
                remaining['fats'] -= food.get('fats', 0)

        # Then add carbs and fats
        carb_foods = suitable_foods.sort_values('carbs', ascending=False)
        for _, food in carb_foods.iterrows():
            if remaining['carbs'] <= 0 and remaining['calories'] <= 0:
                break
            if food['description'] not in selected_foods and food.get('carbs', 0) > 0:
                selected_foods.append(food['description'])
                remaining['carbs'] -= food.get('carbs', 0)
                remaining['calories'] -= food.get('calories', 0)

        meal_plan[meal] = {
            'foods': selected_foods,
            'estimated_nutrition': {
                'calories': meal_targets['calories'] - remaining['calories'],
                'protein': meal_targets['protein'] - remaining['protein'],
                'carbs': meal_targets['carbs'] - remaining['carbs'],
                'fats': meal_targets['fats'] - remaining['fats']
            }
        }

    return meal_plan

# %%
def explain_nutrition_choice(food_item, user_profile):
    """Generate an explanation for a food recommendation"""
    nutrition = get_nutritional_info(food_item)

    prompt = f"""
    Explain why {food_item} (Category: {nutrition['category']}) is recommended for:
    - Health goal: {user_profile['health_goal']}
    - Dietary preference: {user_profile['dietary_preferences']}
    - Allergies: {user_profile['allergies']}

    Nutritional facts (per 100g):
    - Calories: {nutrition['calories']}
    - Protein: {nutrition['protein']}g
    - Carbs: {nutrition['carbs']}g
    - Fats: {nutrition['fats']}g

    Provide a concise, scientific explanation focusing on the health benefits:
    """

    inputs = nutrition_tokenizer(prompt, return_tensors="pt")
    outputs = nutrition_model.generate(**inputs, max_length=200)
    explanation = nutrition_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return explanation

# %%
def suggest_healthier_alternative(current_food, user_profile):
    """Suggest a healthier alternative food"""
    current_nutrition = get_nutritional_info(current_food)

    # Find similar foods with better nutritional profile
    suitable_foods = nutrition_df.copy()

    # Filter based on user profile
    if user_profile['dietary_preferences'] == 'vegetarian':
        suitable_foods = suitable_foods[~suitable_foods['foodCategory'].str.contains('Protein Foods|Meat')]
    if user_profile['allergies'] != 'none':
        suitable_foods = suitable_foods[~suitable_foods['allergens'].str.contains(user_profile['allergies'])]

    # Define what "healthier" means based on health goal
    if user_profile['health_goal'] == 'weight_loss':
        suitable_foods = suitable_foods[
            (suitable_foods['calories'] < current_nutrition['calories']) &
            (suitable_foods['protein'] >= current_nutrition['protein'])
        ]
    elif user_profile['health_goal'] == 'muscle_gain':
        suitable_foods = suitable_foods[
            (suitable_foods['protein'] > current_nutrition['protein']) &
            (suitable_foods['calories'] <= current_nutrition['calories'] * 1.2)
        ]
    else:  # General health
        suitable_foods = suitable_foods[
            (suitable_foods['fats'] < current_nutrition['fats']) &
            (suitable_foods['carbs'] <= current_nutrition['carbs'])
        ]

    if len(suitable_foods) == 0:
        return None

    # Find most similar food that meets criteria
    suitable_foods['similarity'] = suitable_foods['description'].apply(
        lambda x: food_encoder.encode(x.lower()) @
        food_encoder.encode(current_food.lower()).T
    )

    best_alternative = suitable_foods.loc[suitable_foods['similarity'].idxmax()]

    return {
        'alternative': best_alternative['description'],
        'nutrition_comparison': {
            'current': current_nutrition,
            'suggested': {
                'calories': best_alternative.get('calories', 0),
                'protein': best_alternative.get('protein', 0),
                'carbs': best_alternative.get('carbs', 0),
                'fats': best_alternative.get('fats', 0)
            }
        },
        'explanation': explain_nutrition_choice(best_alternative['description'], user_profile)
    }

In [10]:
def analyze_food_image(image):
    """Analyze food from image"""
    food_name = recognize_food_from_image(image)
    nutrition = get_nutritional_info(food_name)
    explanation = explain_nutrition_choice(food_name, {
        'health_goal': 'general',
        'dietary_preferences': 'omnivore',
        'allergies': 'none'
    })

    return {
        "identified_food": food_name,
        "nutritional_info": nutrition,
        "health_analysis": explanation
    }

# %%
def generate_plan_interface(health_goal, dietary_pref, allergies, activity_level):
    """Interface for generating meal plans"""
    plan = generate_meal_plan({
        'health_goal': health_goal,
        'dietary_preferences': dietary_pref,
        'allergies': allergies,
        'activity_level': float(activity_level)
    })

    output = f"## Personalized Meal Plan for {health_goal.replace('_', ' ').title()}\n\n"
    for meal, details in plan.items():
        output += f"### {meal.title()}\n"
        output += "**Foods:** " + ", ".join(details['foods']) + "\n"
        output += "**Nutrition:**\n"
        for nutr, val in details['estimated_nutrition'].items():
            output += f"- {nutr}: {val:.1f}g\n"
        output += "\n"

    return output

# %%
# Create Gradio interface
with gr.Blocks(title="AI Nutrition Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🍏 The Smartest AI Nutrition Assistant")
    gr.Markdown("Get personalized nutrition guidance powered by state-of-the-art AI")

    with gr.Tabs():
        with gr.TabItem("🔍 Food Analysis"):
            with gr.Row():
                image_input = gr.Image(label="Take or upload food photo", type="pil")
                analysis_output = gr.JSON(label="Analysis Results")
            analyze_btn = gr.Button("Analyze Food")
            analyze_btn.click(analyze_food_image, inputs=image_input, outputs=analysis_output)

        with gr.TabItem("📝 Meal Plan Generator"):
            with gr.Row():
                with gr.Column():
                    health_goal = gr.Dropdown(
                        ["weight_loss", "muscle_gain", "maintenance", "diabetic"],
                        label="Your Health Goal",
                        value="maintenance"
                    )
                    dietary_pref = gr.Dropdown(
                        ["omnivore", "vegetarian", "vegan", "pescatarian"],
                        label="Dietary Preference",
                        value="omnivore"
                    )
                    allergies = gr.Dropdown(
                        ["none", "dairy", "gluten", "nuts", "shellfish"],
                        label="Allergies",
                        value="none"
                    )
                    activity_level = gr.Slider(0.8, 1.5, value=1.2, label="Activity Level")
                plan_output = gr.Markdown(label="Your Meal Plan")
            generate_btn = gr.Button("Generate Plan", variant="primary")
            generate_btn.click(
                generate_plan_interface,
                inputs=[health_goal, dietary_pref, allergies, activity_level],
                outputs=plan_output
            )

        with gr.TabItem("🔄 Healthier Alternatives"):
            with gr.Row():
                with gr.Column():
                    current_food = gr.Textbox(label="Current Food Item", placeholder="e.g., white bread")
                    swap_health_goal = gr.Dropdown(
                        ["weight_loss", "muscle_gain", "maintenance", "diabetic"],
                        label="Your Health Goal",
                        value="maintenance"
                    )
                    swap_pref = gr.Dropdown(
                        ["omnivore", "vegetarian", "vegan", "pescatarian"],
                        label="Dietary Preference",
                        value="omnivore"
                    )
                swap_output = gr.JSON(label="Suggested Alternative")
            suggest_btn = gr.Button("Find Healthier Option")
            suggest_btn.click(
                lambda food, goal, pref: suggest_healthier_alternative(
                    food,
                    {'health_goal': goal, 'dietary_preferences': pref, 'allergies': 'none'}
                ),
                inputs=[current_food, swap_health_goal, swap_pref],
                outputs=swap_output
            )


# Launch the interface
demo.launch(debug=True)
class NutritionTracker:
    """Class to track user nutrition and provide feedback"""

    def __init__(self):
        self.user_data = {}
        self.meal_history = {}

    def add_user(self, user_id, initial_profile):
        """Add a new user"""
        self.user_data[user_id] = initial_profile
        self.meal_history[user_id] = []

    def log_meal(self, user_id, meal_type, foods):
        """Log a consumed meal"""
        total_nutrition = {'calories': 0, 'protein': 0, 'carbs': 0, 'fats': 0}

        for food in foods:
            nutrition = get_nutritional_info(food)
            for nutr in total_nutrition:
                total_nutrition[nutr] += nutrition.get(nutr, 0)

        self.meal_history[user_id].append({
            'meal_type': meal_type,
            'foods': foods,
            'nutrition': total_nutrition,
            'timestamp': pd.Timestamp.now()
        })

    def get_daily_summary(self, user_id):
        """Get summary of today's nutrition"""
        today = pd.Timestamp.now().normalize()
        today_meals = [
            m for m in self.meal_history[user_id]
            if m['timestamp'].normalize() == today
        ]

        if not today_meals:
            return "No meals logged today"

        totals = {'calories': 0, 'protein': 0, 'carbs': 0, 'fats': 0}
        for meal in today_meals:
            for nutr in totals:
                totals[nutr] += meal['nutrition'][nutr]

        # Compare to targets
        targets = self._calculate_targets(user_id)
        comparison = {
            nutr: {
                'consumed': totals[nutr],
                'target': targets[nutr],
                'percentage': (totals[nutr] / targets[nutr]) * 100
            }
            for nutr in totals
        }

        return comparison

    def _calculate_targets(self, user_id):
        """Calculate daily nutritional targets"""
        profile = self.user_data[user_id]
        guidelines = {
            'weight_loss': {'calories': 1500, 'protein': 75, 'carbs': 150, 'fats': 50},
            'muscle_gain': {'calories': 2500, 'protein': 150, 'carbs': 300, 'fats': 80},
            'maintenance': {'calories': 2000, 'protein': 50, 'carbs': 250, 'fats': 70},
            'diabetic': {'calories': 1800, 'protein': 60, 'carbs': 150, 'fats': 60}
        }

        targets = guidelines.get(profile['health_goal'], guidelines['maintenance']).copy()
        return {k: v * profile['activity_level'] for k, v in targets.items()}

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1e9ca0e722d70751ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1e9ca0e722d70751ea.gradio.live
